In [ ]:
pip install gdown --user


In [ ]:
pip install numpy --user


In [ ]:
pip install pandas --user


In [4]:
import gdown
url = 'https://drive.google.com/uc?id=14aHI3myaq7v8nv-a87BB9RgJFSQW6ige'
gdown.download(url, 'doge.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=14aHI3myaq7v8nv-a87BB9RgJFSQW6ige
From (redirected): https://drive.google.com/uc?id=14aHI3myaq7v8nv-a87BB9RgJFSQW6ige&confirm=t&uuid=6b017e64-b255-4942-b345-b6881444dcc7
To: c:\Users\vlad\Documents\cmf\doge.csv
100%|██████████| 241M/241M [00:28<00:00, 8.46MB/s] 


'doge.csv'

In [1]:
import pandas as pd
import numpy as np

In [24]:
doges=pd.read_csv('doge.csv',names=['timestamp','side','price','amount'],dtype={"timestamp": int, "side": "string","price":float,"amount":int},skiprows=1)

In [3]:

def calculate_sharpe_ratio(pnl, risk_free_rate=0):
    # 1. Рассчитать доходность как разницу PnL между двумя последовательными интервалами
    returns = np.diff(pnl)

    # 2. Рассчитать среднюю доходность
    mean_return = np.mean(returns)

    # 3. Рассчитать стандартное отклонение доходности
    std_dev_return = np.std(returns)

    # 4. Рассчитать коэффициент Шарпа
    if std_dev_return != 0:
        sharpe_ratio = (mean_return - risk_free_rate) / std_dev_return
    else:
        sharpe_ratio = 0  # Если нет вариации в доходности, Sharpe Ratio будет нулевым

    return sharpe_ratio

# Пример использования:
pnl = [100, 105, 102, 110, 108]  # Пример значений PnL по временным интервалам
sharpe_ratio = calculate_sharpe_ratio(pnl)
print(f'Sharpe Ratio: {sharpe_ratio}')

Sharpe Ratio: 0.43133109281375365


In [4]:
# Function to calculate Sortino Ratio
def calculate_sortino_ratio(portfolio_returns, risk_free_rate=0, target_return=0):
    # Calculate average return of the portfolio
    avg_return = sum(portfolio_returns) / len(portfolio_returns)

    # Calculate the excess return (portfolio return - risk-free rate)
    excess_return = avg_return - risk_free_rate

    # Calculate downside deviations (only returns below target)
    downside_deviation_squared = [(target_return - r) ** 2 for r in portfolio_returns if r < target_return]

    # If there are no downside deviations, the Sortino Ratio can't be calculated
    if len(downside_deviation_squared) == 0:
        return None

    # Calculate the downside deviation
    downside_deviation = (sum(downside_deviation_squared) / len(downside_deviation_squared)) ** 0.5

    # Calculate the Sortino Ratio
    sortino_ratio_value = excess_return / downside_deviation if downside_deviation != 0 else None  # Handle case if downside_deviation is zero

    return sortino_ratio_value

In [5]:
def max_drawdown(prices):
    # Calculate the cumulative maximum of prices
    peak = np.maximum.accumulate(prices)

    # Calculate drawdowns as a percentage
    drawdowns = (peak - prices) / peak

    # Find the maximum drawdown
    max_dd = np.max(drawdowns)

    return max_dd

In [49]:
def calculate_position_flips(positions):
    flips = 0
    rise=False
    if(positions[0]<positions[1]):
        rise=True
    for i in range(1, len(positions)):
        # Check if there's a flip from 'Long' to 'Short' or 'Short' to 'Long'
        if (positions[i] >positions[i-1] and not rise) or \
           (positions[i] <positions[i-1] and rise):
            flips += 1
            rise=not rise
    
    return flips

In [48]:


class MarketSimulator:
    def __init__(self, order_books, trades, window_size):
        self.order_books = order_books
        self.trades = trades
        self.window_size = window_size
        self.candles = {}
    def get_candles(self):
        return self.candles
    def generate_candles(self):
        for instrument, trades in self.trades.items():
            candles = []
            current_window = []
            start_time = trades.values[0][0]

            for i,trade in trades.iterrows():
                if trade['timestamp'] - start_time < self.window_size:
                    current_window.append(trade)
                else:
                    if current_window:
                        candle = self.calculate_candle(current_window)
                        candles.append(candle)
                    current_window = [trade]
                    start_time = trade['timestamp']

            if current_window:
                candle = self.calculate_candle(current_window)
                candles.append(candle)

            self.candles[instrument] = candles

    def calculate_candle(self, trades):
        open_price = trades[0]['price']
        close_price = trades[-1]['price']
        low_price = min(trade['price'] for trade in trades)
        high_price = max(trade['price'] for trade in trades)
        buy_trades = [trade for trade in trades if trade['side'] == 'buy']
        sell_trades = [trade for trade in trades if trade['side'] == 'sell']
        avg_buy_price = np.mean([trade['price'] for trade in buy_trades]) if buy_trades else None
        avg_sell_price = np.mean([trade['price'] for trade in sell_trades]) if sell_trades else None
        buy_volume = sum(trade['amount'] for trade in buy_trades)
        sell_volume = sum(trade['amount'] for trade in sell_trades)

        return {
            'open': open_price,
            'close': close_price,
            'low': low_price,
            'high': high_price,
            'avg_buy_price': avg_buy_price,
            'avg_sell_price': avg_sell_price,
            'buy_volume': buy_volume,
            'sell_volume': sell_volume
        }

    def run_simulation(self, strategies):
        results = []
        base_info={}
        for instrument, candles in self.candles.items():
          max_dd=max_drawdown([candle['close'] for candle in candles])
          flips=calculate_position_flips([candle['close'] for candle in candles])
          base_info[instrument]={}
          base_info[instrument]["max_dd"]=max_dd
          base_info[instrument]["flips"]=flips
        for strategy in strategies:
            result = self.simulate_strategy(strategy)
            result["base_info"]=base_info
            results.append(result)
        return results

    def simulate_strategy(self, strategy):
        pnl = 0
        amount = 0
        stats={}
        for instrument, actions in strategy.items():
            candles = self.candles[instrument]
            stat={
                'traded_volume':0,
                'pnl':[],
                'sharpe_ratio':0,
                'sortino_ratio':0,
                'average_holding_time':0
            }
            aht=0
            aht_max=-1
            for i, action in enumerate(actions):

                candle = candles[i]
                if action > 0:
                    # Покупка
                    execution_price = candle['avg_buy_price'] if candle['avg_buy_price'] else candle['close']
                    pnl -= action * execution_price
                    amount += action
                    stat['pnl'].append(-action * execution_price)
                    stat['traded_volume']+=action
                    aht+=self.window_size
                    if(aht>aht_max):
                        aht_max=aht
                elif action < 0:
                    # Продажа
                    if(amount-action>=0):
                        execution_price = candle['avg_sell_price'] if candle['avg_sell_price'] else candle['close']
                        pnl += abs(action) * execution_price
                        amount -= action
                        stat['pnl'].append(abs(action) * execution_price)
                        stat['traded_volume']+=abs(action)
                        aht=0
            stat['sharpe_ratio']=calculate_sharpe_ratio(stat['pnl'])
            stat['sortino_ratio']=calculate_sortino_ratio(stat['pnl'])
            print(stat['pnl'])
            stat['pnl']=sum(stat['pnl'])
            stat['average_holding_time']=max(aht_max,aht)
            stats[instrument]=stat
        return stats

# Пример использования:
order_books = {}  # данные ордербуков

# strategies = [
#     {'DOGEUSDT': [-1, 1]},
# ]

# results = simulator.run_simulation(strategies)
# print(results)


In [50]:
from numpy.random import seed
from numpy.random import randint
# seed random number generator
seed(12)
trades = {
    'DOGEUSDT': doges
}

simulator = MarketSimulator(order_books, trades, window_size=1000*60*60*24)
simulator.generate_candles()
candles=simulator.get_candles()


random_strategy=randint(-100, 100, len(candles['DOGEUSDT']))
strategies=[
    {'DOGEUSDT':random_strategy}
]
results = simulator.run_simulation(strategies)
print(results)

[np.float64(2.5915628029079154), np.float64(-5.699472988505748), np.float64(-3.5211663716814163), np.float64(-3.1048379562043795), np.float64(10.06285947858473), np.float64(-9.851428686327075), np.float64(2.4850162725450904), np.float64(-7.894040894941634), np.float64(8.108870504347827), np.float64(5.303590130890052), np.float64(-8.325458585858586), np.float64(-3.436647845601436), np.float64(-4.271286875), np.float64(-6.467011538461539), np.float64(-1.879940781818182), np.float64(2.7244668813825608), np.float64(-2.948101451804839), np.float64(-1.7845109257375382), np.float64(-0.41876097777777777), np.float64(9.087003003754694), np.float64(1.8793636165048544), np.float64(1.1448712740384615), np.float64(6.747035153061225), np.float64(-9.350119873817034), np.float64(-6.542546193353474), np.float64(-6.321186698473283), np.float64(-6.009220471512769), np.float64(-1.9686302649006624), np.float64(-9.424622108963094), np.float64(0.41374902654867257), np.float64(-4.768339925187033), np.float64(